<a href="https://colab.research.google.com/github/som1966/Final-Project/blob/zohairBranch/finalProject_firstsegment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

reading in files from data storage so that we can manipulate them

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

First, enter the following code to download a Postgres driver that will allow Spark to interact with Postgres:

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar


--2022-01-17 13:34:00--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.12’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-01-17 13:34:01 (6.56 MB/s) - ‘postgresql-42.2.16.jar.12’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FinalProject-ETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

performed the  first two steps of the ETL process before pyspark

In [4]:
# connect to data storage --- Extract 
# Read in data from S3 Buckets
from pyspark import SparkFiles
url1 ="https://corti-gw-group-project.s3.amazonaws.com/Unprocessed+Data/Happiness_Scores_2019.csv"
spark.sparkContext.addFile(url1)
Happiness_Scores_2019_df = spark.read.csv(SparkFiles.get("Happiness_Scores_2019.csv"), sep=",", header=True, inferSchema=True)
Happiness_Scores_2019_df = Happiness_Scores_2019_df.drop('yr') \
       .drop('country_name')

Happiness_Scores_2019_df.show()

+------------+-----------+
|country_code|Life Ladder|
+------------+-----------+
|           4|      2.375|
|           8|      4.995|
|          12|      4.745|
|          32|      6.086|
|          51|      5.488|
|          36|      7.234|
|          40|      7.195|
|          31|      5.173|
|          48|      7.098|
|          50|      5.114|
|         112|      5.821|
|          56|      6.772|
|         204|      4.976|
|          68|      5.674|
|          70|      6.016|
|          72|      3.471|
|          76|      6.451|
|         100|      5.108|
|         854|      4.741|
|         116|      4.998|
+------------+-----------+
only showing top 20 rows



In [5]:
from pyspark import SparkFiles
url2 ="https://corti-gw-group-project.s3.amazonaws.com/Unprocessed+Data/UN_Consumer_Price+Index_2019.csv"
spark.sparkContext.addFile(url2)
UN_Consumer_Price_Index_2019_df = spark.read.csv(SparkFiles.get("UN_Consumer_Price+Index_2019.csv"), sep=",", header=True, inferSchema=True)

#Remove multiple columnss in one go
UN_Consumer_Price_Index_2019_df = UN_Consumer_Price_Index_2019_df.drop('Year')
UN_Consumer_Price_Index_2019_df = UN_Consumer_Price_Index_2019_df.drop('_c1')\
        .drop('series')
UN_Consumer_Price_Index_2019_df=UN_Consumer_Price_Index_2019_df.withColumnRenamed("Value","consumer_price_index")

UN_Consumer_Price_Index_2019_df.show(truncate=False)


+-------------------+--------------------+
|Region/Country/Area|consumer_price_index|
+-------------------+--------------------+
|4                  |149.9               |
|8                  |119.1               |
|12                 |151.4               |
|20                 |105.8               |
|24                 |378.9               |
|660                |107.1               |
|28                 |115.4               |
|32                 |232.8               |
|51                 |129.2               |
|533                |109.2               |
|36                 |119.8               |
|40                 |118.1               |
|31                 |156.9               |
|44                 |116.2               |
|48                 |118.8               |
|50                 |179.7               |
|52                 |191.7               |
|112                |508.1               |
|56                 |117.1               |
|84                 |106.2               |
+----------

In [6]:
url3 ="https://corti-gw-group-project.s3.amazonaws.com/Unprocessed+Data/UN_Country.csv"
spark.sparkContext.addFile(url3)
UN_Country_df = spark.read.csv(SparkFiles.get("UN_Country.csv"), sep=",", header=True, inferSchema=True)
UN_Country_df.show()

+------------+-------------------+
|country_code|       country_name|
+------------+-------------------+
|           4|        Afghanistan|
|           8|            Albania|
|          12|            Algeria|
|          16|     American Samoa|
|          20|            Andorra|
|          24|             Angola|
|         660|           Anguilla|
|          28|Antigua and Barbuda|
|          32|          Argentina|
|          51|            Armenia|
|         533|              Aruba|
|          36|          Australia|
|          40|            Austria|
|          31|         Azerbaijan|
|          44|            Bahamas|
|          48|            Bahrain|
|          50|         Bangladesh|
|          52|           Barbados|
|         112|            Belarus|
|          56|            Belgium|
+------------+-------------------+
only showing top 20 rows



In [7]:
url4 ="https://corti-gw-group-project.s3.amazonaws.com/Unprocessed+Data/UN_GDP_Per_Capita_2019.csv"
spark.sparkContext.addFile(url4)
UN_GDP_Per_Capita_2019_df = spark.read.csv(SparkFiles.get("UN_GDP_Per_Capita_2019.csv"), sep=",", header=True, inferSchema=True)

#Remove multiple columnss in one go
UN_GDP_Per_Capita_2019_df = UN_GDP_Per_Capita_2019_df.drop('Year')
UN_GDP_Per_Capita_2019_df = UN_GDP_Per_Capita_2019_df.drop('_c1')\
        .drop('series')
UN_GDP_Per_Capita_2019_df=UN_GDP_Per_Capita_2019_df.withColumnRenamed("Value","per_capita_gdp_dollars")

UN_GDP_Per_Capita_2019_df.show(truncate=False)

+-------------------+----------------------+
|Region/Country/Area|per_capita_gdp_dollars|
+-------------------+----------------------+
|4                  |$470.00               |
|8                  |$5,303.00             |
|12                 |$3,976.00             |
|20                 |$40,887.00            |
|24                 |$2,671.00             |
|660                |$25,529.00            |
|28                 |$17,113.00            |
|32                 |$10,041.00            |
|51                 |$4,623.00             |
|533                |$30,975.00            |
|36                 |$54,763.00            |
|40                 |$49,701.00            |
|31                 |$4,782.00             |
|44                 |$34,864.00            |
|48                 |$23,504.00            |
|50                 |$1,846.00             |
|52                 |$18,149.00            |
|112                |$6,674.00             |
|56                 |$46,198.00            |
|84       

In [8]:
url5 ="https://corti-gw-group-project.s3.amazonaws.com/Unprocessed+Data/UN_Gender_Ratio_2019.csv"
spark.sparkContext.addFile(url5)
UN_Gender_Ratio_2019_df = spark.read.csv(SparkFiles.get("UN_Gender_Ratio_2019.csv"), sep=",", header=True, inferSchema=True)

#Remove multiple columnss in one go
UN_Gender_Ratio_2019_df = UN_Gender_Ratio_2019_df.drop('Year')
UN_Gender_Ratio_2019_df = UN_Gender_Ratio_2019_df.drop('_c1')\
        .drop('series')
UN_Gender_Ratio_2019_df=UN_Gender_Ratio_2019_df.withColumnRenamed("Value","gender_ratio_males_per100_female")

UN_Gender_Ratio_2019_df.show(truncate=False)

+-------------------+--------------------------------+
|Region/Country/Area|gender_ratio_males_per100_female|
+-------------------+--------------------------------+
|4                  |105.5                           |
|8                  |103.7                           |
|12                 |102.1                           |
|24                 |97.9                            |
|28                 |93.2                            |
|32                 |95.2                            |
|51                 |88.8                            |
|533                |90.3                            |
|36                 |99.2                            |
|40                 |97.0                            |
|31                 |99.7                            |
|44                 |94.5                            |
|48                 |179.9                           |
|50                 |102.4                           |
|52                 |93.7                            |
|112      

In [9]:
url6 ="https://corti-gw-group-project.s3.amazonaws.com/Unprocessed+Data/UN_Infant_Mortality_2020.csv"
spark.sparkContext.addFile(url6)
UN_Infant_Mortality_2020_df = spark.read.csv(SparkFiles.get("UN_Infant_Mortality_2020.csv"), sep=",", header=True, inferSchema=True)

#Remove multiple columnss in one go
UN_Infant_Mortality_2020_df = UN_Infant_Mortality_2020_df.drop('Year')
UN_Infant_Mortality_2020_df = UN_Infant_Mortality_2020_df.drop('_c1')\
        .drop('series')
UN_Infant_Mortality_2020_df=UN_Infant_Mortality_2020_df.withColumnRenamed("Value","infant_mortality_per1000_births")
UN_Infant_Mortality_2020_df.show(truncate=False)

+-------------------+-------------------------------+
|Region/Country/Area|infant_mortality_per1000_births|
+-------------------+-------------------------------+
|4                  |51.7                           |
|8                  |8.0                            |
|12                 |21.2                           |
|24                 |61.5                           |
|28                 |5.2                            |
|32                 |10.2                           |
|51                 |10.8                           |
|533                |13.6                           |
|36                 |3.1                            |
|40                 |3.2                            |
|31                 |20.8                           |
|44                 |5.9                            |
|48                 |6.0                            |
|50                 |26.8                           |
|52                 |10.0                           |
|112                |3.0    

In [10]:

url7 ="https://corti-gw-group-project.s3.amazonaws.com/Unprocessed+Data/UN_Life_Expectancy_2020.csv"
spark.sparkContext.addFile(url7)
UN_Life_Expectancy_2020_df = spark.read.csv(SparkFiles.get("UN_Life_Expectancy_2020.csv"), sep=",", header=True, inferSchema=True)
UN_Life_Expectancy_2020_df = UN_Life_Expectancy_2020_df.drop('Year')
UN_Life_Expectancy_2020_df = UN_Life_Expectancy_2020_df.drop('_c1')\
        .drop('series')
UN_Life_Expectancy_2020_df=UN_Life_Expectancy_2020_df.withColumnRenamed("Value","Life_Expectancy")

UN_Life_Expectancy_2020_df.show()

+-------------------+---------------+
|Region/Country/Area|Life_Expectancy|
+-------------------+---------------+
|                  4|           64.3|
|                  8|           78.4|
|                 12|           76.6|
|                 24|           60.5|
|                 28|           76.8|
|                 32|           76.4|
|                 51|           74.9|
|                533|           76.1|
|                 36|           83.2|
|                 40|           81.4|
|                 31|           72.8|
|                 44|           73.7|
|                 48|           77.1|
|                 50|           72.2|
|                 52|           79.0|
|                112|           74.5|
|                 56|           81.4|
|                 84|           74.4|
|                204|           61.3|
|                 64|           71.3|
+-------------------+---------------+
only showing top 20 rows



In [11]:
url8 ="https://corti-gw-group-project.s3.amazonaws.com/Unprocessed+Data/UN_Population_Density_2019.csv"
spark.sparkContext.addFile(url8)
table8_df = spark.read.csv(SparkFiles.get("UN_Population_Density_2019.csv"), sep=",", header=True, inferSchema=True)

table8_df = table8_df.drop('Year')
table8_df = table8_df.drop('_c1')\
        .drop('series')
table8_df=table8_df.withColumnRenamed("Value","population_density")

table8_df.show(truncate=False)


+-------------------+------------------+
|Region/Country/Area|population_density|
+-------------------+------------------+
|4                  |58.3              |
|8                  |105.1             |
|12                 |18.1              |
|16                 |276.6             |
|20                 |164.1             |
|24                 |25.5              |
|660                |165.2             |
|28                 |220.7             |
|32                 |16.4              |
|51                 |103.9             |
|533                |590.6             |
|36                 |3.3               |
|40                 |108.7             |
|31                 |121.6             |
|44                 |38.9              |
|48                 |2159.4            |
|50                 |1252.6            |
|52                 |667.5             |
|112                |46.6              |
|56                 |381.1             |
+-------------------+------------------+
only showing top

In [12]:
url9 ="https://corti-gw-group-project.s3.amazonaws.com/Unprocessed+Data/UN_Seats_Held_by_Women_2019.csv"
spark.sparkContext.addFile(url9)
table9_df = spark.read.csv(SparkFiles.get("UN_Seats_Held_by_Women_2019.csv"), sep=",", header=True, inferSchema=True)

table9_df = table9_df.drop('series')\
        .drop('Year')\
        .drop('_c1')\
        .drop('Last Election Date')\
        .drop('Last Election Date footnote')
table9_df=table9_df.withColumnRenamed("Value","seats_held_by_woman_pct")

table9_df.show(truncate=False)

+-------------------+-----------------------+
|Region/Country/Area|seats_held_by_woman_pct|
+-------------------+-----------------------+
|8                  |29.3                   |
|12                 |25.8                   |
|20                 |32.1                   |
|24                 |30.0                   |
|28                 |11.1                   |
|32                 |38.8                   |
|51                 |24.2                   |
|36                 |30.0                   |
|40                 |37.2                   |
|31                 |16.8                   |
|44                 |12.8                   |
|48                 |15.0                   |
|50                 |20.7                   |
|52                 |20.0                   |
|112                |34.5                   |
|56                 |38.0                   |
|84                 |9.4                    |
|204                |7.2                    |
|64                 |14.9         

In [13]:
url10 ="https://corti-gw-group-project.s3.amazonaws.com/Unprocessed+Data/UN_Unemployment_Rate_2020.csv"
spark.sparkContext.addFile(url10)
table10_df = spark.read.csv(SparkFiles.get("UN_Unemployment_Rate_2020.csv"), sep=",", header=True, inferSchema=True)

table10_df = table10_df.drop('_c1')\
        .drop('series')\
        .drop('Last Election Date')\
        .drop('Year')\
        .drop('Last Election Date footnote')
table10_df=table10_df.withColumnRenamed("Value","unemployment_rate")

table10_df.show(truncate=False)

+-------------------+-----------------+
|Region/Country/Area|unemployment_rate|
+-------------------+-----------------+
|4                  |11.2             |
|8                  |12.8             |
|12                 |11.5             |
|24                 |6.8              |
|32                 |10.4             |
|51                 |16.6             |
|36                 |5.3              |
|40                 |4.8              |
|31                 |6.0              |
|44                 |11.3             |
|48                 |0.8              |
|50                 |4.2              |
|52                 |10.9             |
|112                |4.6              |
|56                 |5.7              |
|84                 |6.4              |
|204                |2.0              |
|64                 |2.4              |
|68                 |3.5              |
|70                 |18.4             |
+-------------------+-----------------+
only showing top 20 rows



In [14]:
url11 ="https://corti-gw-group-project.s3.amazonaws.com/Unprocessed+Data/UN_Water_Services_2020.csv"
spark.sparkContext.addFile(url11)
table11_df = spark.read.csv(SparkFiles.get("UN_Water_Services_2020.csv"), sep=",", header=True, inferSchema=True)

table11_df = table11_df.drop('_c1')\
        .drop('series')\
        .drop('Last Election Date')\
        .drop('Year')\
        .drop('Last Election Date footnote')
table11_df=table11_df.withColumnRenamed("Value","safe_drinking_water_access_pct")

table11_df.show(truncate=False)

+-------------------+------------------------------+
|Region/Country/Area|safe_drinking_water_access_pct|
+-------------------+------------------------------+
|4                  |27.6                          |
|8                  |70.7                          |
|12                 |72.4                          |
|16                 |98.4                          |
|20                 |90.6                          |
|51                 |86.9                          |
|40                 |98.9                          |
|31                 |88.3                          |
|48                 |99.0                          |
|50                 |58.5                          |
|112                |94.6                          |
|56                 |99.9                          |
|64                 |36.6                          |
|70                 |88.9                          |
|76                 |85.8                          |
|100                |97.6                     

In [17]:

url12 ="https://zohair-gw-group-project.s3.amazonaws.com/Unprocessed+Data/Democracy_Index_2019.csv"
spark.sparkContext.addFile(url12)
table12_df = spark.read.csv(SparkFiles.get("Democracy_Index_2019.csv"), sep=",", header=True, inferSchema=True)

table12_df = table12_df.drop('country_name')\
        .drop('yr')
table12_df=table12_df.withColumnRenamed("Democracy index (EIU)","democracy_index")

table12_df.show(truncate=False)

AnalysisException: ignored

In [21]:
# joined the two dataframes
joined1_df= UN_Country_df.join(Happiness_Scores_2019_df, on="country_code", how="inner")
joined1_df.show()

Py4JJavaError: ignored

In [ ]:
# Transform


https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/

https://sparkbyexamples.com/pyspark/pyspark-rename-dataframe-column/


https://jacobcelestine.com/knowledge_repo/colab_and_pyspark/

